In [ ]:
!pip install pydub
!pip install gtts
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors
!pip install edge-tts
!pip install mutagen

In [ ]:
import google.generativeai as genai
import torch
from torchvision.utils import save_image
from moviepy.editor import ImageSequenceClip
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips,AudioFileClip
from gtts import gTTS


In [ ]:
genai.configure(api_key="Use your gemini api key here")
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
def chat(prompt):
    response = model.generate_content(prompt)
    return(response.text)

In [ ]:
#chat("What is AI")

In [ ]:
# Step 3: Load pre-trained model
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe = pipe.to("cuda")

In [ ]:
start = 3

In [ ]:
if start == 1:
  # 1. Story telling
  context="""You are a professional story teller skilled in adventure, scifi, comedy, fiction, magic, romance, thriller and scary stories.
You must tell the stories in less than 200 words and also stick to the theme of the story.
Optionaly you can include anime/cartoon characters. The user will input [adventure, scifi, comedy, fiction, magic, romance, thriller, scary]
any topic from these and you have to make a story around it and remember to make it in less than 200 words, also the user can input a combination of different topics like [fiction, scary, adventure] and you have to make a story considering all the topics.
The following is the topic from the user: """
elif start == 2:
  # 2. Productivity coach
  context = """ You are a retired ceo of multinational tech company who gives advice/tips on any of the given topics ['financial independence', 'productivity', 'time management', 'self improvement', 'mental peace'].
The user will input ['financial independence', 'productivity', 'time management', 'self improvement', 'mental peace'] any topic from these and you give a 200 words advice/tips on any one of the one above topic and stick to the same topic throughout the text.
You can optionaly take inspiration from books or blogs of productivity and time management.
The following is the topic from the user: """
elif start == 3:
  # 3. Facts Teller
  context = """ You are geek and nerd in knowing all the facts related to topics like ['tech fact', 'eductional fact', 'funny fact', 'historical fact', 'movie fact', 'scary fact'].
The user will input ['financial independence', 'productivity', 'time management', 'self improvement', 'mental peace'] any topic from these and you give a 200 words fact related to any one of the one above topic and stick to the same topic throughout the text.
You can take inspiration from internet and books to check if the correct or not. Only give correct facts.
The following is the topic from the user: """

In [ ]:
if start == 1:
  # 1. Story telling
  a="""You are professional prompt engineer task with converting a text story into image generation prompts which can be used to generate images corresponding whats happening in the story.
Put each prompt following the sequence of the story in a Python string format separated by ';' and don't use single or double quotes. Don't add 'python\nprompts = ' or 'prompts = ' at the start and also dont give '\n' in the prompts just give the prompts.
Create exactly 20 prompts to generate 20 images for the story. Take the general idea of whats happening in the story and create elobrate prompts for the story.
Make the prompts in meaningful sentences which stable diffusion model can understand and generate images.
Following is the story: """
elif start == 2:
  # 2. Productivity coach
  a="""You are professional prompt engineer task with converting a text productivity advice/tips into image generation prompts which can be used to generate images corresponding whats spoken in the text.
Put each prompt following the sequence of the text in a Python string format separated by ';' and don't use single or double quotes. Don't add 'python\nprompts = ' or 'prompts = ' at the start and also dont give '\n' in the prompts just give the prompts.
Create exactly 20 prompts to generate 20 images for the text. Take the general idea of whats happening in the productivity text and create elobrate prompts for the text.
Make the prompts in meaningful sentences which stable diffusion model can understand and generate images.
Following is the productivity advice/tip text: """
elif start == 3:
  # 3. Facts Teller
  a="""You are professional prompt engineer task with converting a text related to facts into image generation prompts which can be used to generate images corresponding whats spoken in the text.
Put each prompt following the sequence of the text in a Python string format separated by ';' and don't use single or double quotes. Don't add 'python\nprompts = ' or 'prompts = ' at the start and also dont give '\n' in the prompts just give the prompts.
Create exactly 20 prompts to generate 20 images for the text. Take the general idea of whats happening in the text related to facts and create elobrate prompts for the text.
Make the prompts in meaningful sentences which stable diffusion model can understand and generate images.
Following is the fact related text: """

In [ ]:
import random
# Step 2: Parse text data
def parse_text(text):
    # Implement your text parsing logic here
    # Extract relevant descriptions for each scene
    x=chat(a+text)
    txt= x.split(";")
    while len(txt)>20:
      txt.pop(random.randint(0, len(txt) - 1))
    return(txt)


In [ ]:
def generate_images(model, text_descriptions, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    i=100
    for description in text_descriptions:
        # Preprocess the text if needed
        # Feed the text description to the model to generate an image
        generator = torch.Generator("cuda").manual_seed(1024)
        generated_image = model(description, generator=generator, height=1280, width=720).images[0]
        i+=1
        #generated_image = model.generate_image(description)
        # Save the generated image
        #save_image(generated_image, f"{description[:20]}.png")  # Saving with the first 20 characters of the description as filename
        generated_image.save(os.path.join(output_folder, f"{str(i)}.png"))
        #save_image(generated_image, os.path.join(output_folder, f"{description[:20]}.png"))

In [ ]:
from moviepy.video.VideoClip import ImageClip

# Function to create a video from images with each frame lasting 2 seconds
def create_video(images_folder, output_video_path, duration_per_frame=30):
    # Get list of image files in the folder
    image_files = sorted([os.path.join(images_folder, f) for f in os.listdir(images_folder) if f.endswith('.png')])

    # Create a list of ImageClips with each image lasting for the specified duration
    clips = [ImageClip(img).set_duration(duration_per_frame) for img in image_files]

    # Concatenate all the ImageClips
    video = concatenate_videoclips(clips, method="compose")

    # Write the video file
    video.write_videofile(output_video_path, codec='libx264', fps=30)  # FPS here is for smooth playback


In [ ]:
import edge_tts

def text_to_audio(text, audio_path):
    VOICE = "en-GB-SoniaNeural"
    communicate = edge_tts.Communicate(text, VOICE)
    communicate.save_sync(audio_path)

In [ ]:
# Function to merge audio with video
def merge_audio_video(audio_path, video_path, output_path):
      video = VideoFileClip(video_path)
      audio = AudioFileClip(audio_path)
      video = video.set_audio(audio)
      video.write_videofile(output_path, codec='libx264')

In [ ]:
from pydub import AudioSegment

def final_audio(initial_audio_path):
  # Load the audio file
  audio = AudioSegment.from_file(initial_audio_path)  # Replace with your file path

  # Check the length of the audio
  if len(audio) > 59000:  # 59 seconds in milliseconds
      # Trim the audio to the first 59 seconds
      audio = audio[:59000]

  # Export the trimmed or padded audio
  audio.export("output_audio.mp3", format="mp3")

In [ ]:
import random
if start == 1:
  # 1. Story telling
  topic=['adventure', 'scifi', 'comedy', 'fiction', 'magic', 'romance', 'thriller', 'scary']
  random_topic = random.sample(topic, 1)
elif start == 2:
  # 2. Productivity coach
  topic=['financial independence', 'productivity', 'time management', 'self improvement', 'mental peace']
  random_topic = random.sample(topic, 1)
elif start == 3:
  # 3. Facts Teller
  topic=['tech fact', 'eductional fact', 'funny fact', 'historical fact', 'movie fact', 'scary fact']
  random_topic = random.sample(topic, 1)

In [ ]:
x=''
for i in random_topic:
    x=x+i+" "
print(x)


In [ ]:
text=chat(context+x)
print(text)

In [ ]:
def output(text):

  text_descriptions = parse_text(text)
  #print(text_descriptions)
  # Step 2: Load pre-trained model
  model = pipe
  # Step 3 & 4: Generate images from text
  #generate_images(model, text_descriptions, output_folder)
  output_folder = "output_images"
  generate_images(model, text_descriptions, output_folder)


  # Example usage
  images_folder = "/content/output_images"
  output_video_path = "output_video.mp4"
  create_video(images_folder, output_video_path, duration_per_frame=3)


  initial_audio_path = "initial_output_audio.mp3"
  video_path = "/content/output_video.mp4"
  output_path = "final_output.mp4"
  # Convert text to audio
  #text_to_audio(text, audio_path)
  text_to_audio(text, initial_audio_path)
  final_audio(initial_audio_path)
  audio_path = "output_audio.mp3"

  # Merge audio with video
  merge_audio_video(audio_path, video_path, output_path)
  video_path = "final_output.mp4"
  return video_path

In [ ]:
video_path = output(text)

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video_in_colab(video_path):
    # Encode video to base64
    with open(video_path, "rb") as video_file:
        video_data = video_file.read()
        video_base64 = b64encode(video_data).decode()

    # Create HTML for displaying video
    video_html = f"""
    <video width="640" height="360" controls>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """

    # Display the video
    return HTML(video_html)

# Example usage
video_path = "final_output.mp4" # Replace with the actual video path
play_video_in_colab(video_path)
